<a href="https://colab.research.google.com/github/p76081158/ipc-example/blob/main/python/ipc_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Examples of Inter-Process Communication
* [Inter-Process Communication](https://vcx1127.notion.site/Inter-Process-Communication-4be174067b024db69160f362f3f31668)

![](https://i.imgur.com/IUjClbL.png)
* 總共 4 個 processes (Main、3 Clients)

In [ ]:
from io import StringIO
import threading
import time
import os
import errno
import socket
import sys
import statistics

In [ ]:
# var
FIFO  = 'mypipe'
Shared = None

* 定義 Named Pipes file name 和 Shared Memory

In [ ]:
# get mean value
def caclmean(nums):
    return statistics.mean(nums)

In [ ]:
# get median value
def caclmedian(nums):
    return statistics.median(nums)

In [ ]:
# get mode value
def caclmode(nums):
    return statistics.mode(nums)

In [ ]:
# string to int array
def string_to_array(str):
    return list(map(int, str.split(" ")))

In [ ]:
# tcp socket client
def client1():
    HOST = '127.0.0.1'
    PORT = 8020
    clientMessage = 'Client1 is ready'

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((HOST, PORT))
    client.sendall(clientMessage.encode())

    serverMessage = str(client.recv(1024), encoding='utf-8')
    
    # print Mean value
    print('Mean is', caclmean(string_to_array(serverMessage)))

    client.close()

| 欄位類別 | 名稱宣告 | 解釋內容 |
| --- | --- | --- |
| family | socket.AF_UNIX	 | 於本機端進行串接 |
| --- | socket.AF_INET | 於伺服器與伺服器之間進行串接 |
| --- | socket.AF_INET6 | 使用IPv6於伺服器與伺服器之間進行串接 |
| type | socket.SOCK_STREAM	 | 使用TCP(資料流)的方式提供可靠、雙向、串流的通信頻道 |
| --- | socket.SOCK_DGRAM | 使用UDP()的方式通用的免連線訊息交換通道 |

In [ ]:
# named pipes client
def client2():
    print("Client2 is ready\n")
    while True:
        with open(FIFO) as fifo:
            while True:
                data = fifo.read()
                if len(data) == 0:
                    break
                print('Median is', caclmedian(string_to_array(data)))
                sys.exit()

In [ ]:
# shared memory client
def client3():
    global Shared
    print("Client3 is ready\n")
    while Shared == None:
        continue
    print('Mode is', caclmode(string_to_array(Shared)))

In [ ]:
# tcp server
HOST = '127.0.0.1'
PORT = 8020

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(10)

In [ ]:
# named piped server
FIFO = 'mypipe'
try:
    os.mkfifo(FIFO)
except OSError as oe:
    if oe.errno != errno.EEXIST:
        raise

In [ ]:
# start clients
c1 = threading.Thread(name='client1', target=client1)
c2 = threading.Thread(name='client2', target=client2)
c3 = threading.Thread(name='client3', target=client3)
c1.start()
c2.start()
c3.start()

Client2 is ready

Client3 is ready



* 使用 3 個 Threads 分別執行 Clients

In [ ]:
# tcp server accept client connection
conn, addr = server.accept()
clientMessage = str(conn.recv(1024), encoding='utf-8')

print(clientMessage)

Client1 is ready


In [ ]:
# stdin via StringIO
sys.stdin = StringIO("1 5 5 10 15 2 3\n")
msg       = sys.stdin.readline()

* 透過 StringIO 在 Jupyiter notebook 上模擬 Stdin

In [ ]:
# tcp server send to client1
conn.sendall(msg.encode())
conn.close()

Mean is 5.857142857142857


In [ ]:
# named piped server send to client2
result = os.system('echo {0} > mypipe '.format('"'+msg+'"'))

Median is 5


In [ ]:
# shared memory server update memory
Shared = msg

Mode is 5


In [ ]:
# program ending
print("Ending\n")
c1.join()
c2.join()
c3.join()

Ending

